In [420]:
%run ./1_DataIngestion.ipynb

C:\Users\mike_\AppData\Local\Temp\ipykernel_4136\129421425.py:2: DtypeWarning: Columns (59,60) have mixed types. Specify dtype option on import or set low_memory=False.
  health_locations = pd.read_csv(filename)
C:\Users\mike_\AppData\Local\Temp\ipykernel_4136\3500236455.py:2: DtypeWarning: Columns (35) have mixed types. Specify dtype option on import or set low_memory=False.
  wholesale = pd.read_csv(filename)
C:\Users\mike_\AppData\Local\Temp\ipykernel_4136\2127765340.py:2: DtypeWarning: Columns (35) have mixed types. Specify dtype option on import or set low_memory=False.
  construction = pd.read_csv(filename)
C:\Users\mike_\AppData\Local\Temp\ipykernel_4136\1436630130.py:2: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  goverment = pd.read_csv(filename)


#### 1. Build dataframe with the ENOE foodstand location count aggregated by State and ZipCode 

In [421]:
# Analize frecuency of words inside the type of foodstand
#pd.Series(' '.join(denoe_puestos['nombre_act']).split()).value_counts()

food_types = [
    'tortas',
    'tacos',
    'antojitos',
    'sodas',
    'pizzas',
    'hamburguesas',
    'dogs',
    'mariscos',
    'corrida',
    'pescados'
    ]

# Structure and standardize foodstand types
denoe_puestos['FoodstandType'] = np.where(denoe_puestos.nombre_act.str.contains("tacos"), "Tacos",
                                 np.where(denoe_puestos.nombre_act.str.contains("antojitos"), "Antojitos",
                                 np.where(denoe_puestos.nombre_act.str.contains("pizzas"), "Pizza",
                                 np.where(denoe_puestos.nombre_act.str.contains("hamburguesas"), "Hamburguesas",
                                 np.where(denoe_puestos.nombre_act.str.contains("dogs"), "Hot-Dogs",
                                 np.where(denoe_puestos.nombre_act.str.contains("mariscos"), "Maricos",
                                 np.where(denoe_puestos.nombre_act.str.contains("pescados"), "Maricos",
                                np.where(denoe_puestos.nombre_act.str.contains("corrida"), "Comida Corrida", "Tortas")))))))
                                )

# Aggregate foodstand count by State and ZipCode
foodstandsByLocation = denoe_puestos.groupby(['entidad','cod_postal'])['FoodstandType'].count().reset_index(drop=False)\
                                 .rename(columns={
                                                 'FoodstandType':'FoodstandCount',
                                                 'entidad':'State',
                                                 'cod_postal':'ZipCode'
                                                 }
                                        )

# format zipcodes to string 5 characters
foodstandsByLocation['ZipCode'] = foodstandsByLocation['ZipCode'].astype(str).apply(str).str.zfill(5)


#### 2. Build a dataframe with State, Alcaldia and ZipCode info

In [422]:
# keep state, alcaldia and zipcode columns. drop duplicates
tmp = pd.DataFrame().assign(State = inegi_zip_codes['d_estado'],\
                            Alcaldia = inegi_zip_codes['D_mnpio'],\
                            ZipCode = inegi_zip_codes['d_codigo']\
                            ).drop_duplicates().reset_index(drop=True)

tmp['ZipCode'] = tmp['ZipCode'].apply(str).str.zfill(5) # ZipCode as string, left pad with 0, 5 digits zip codeformat

# keep only a few alcaldias for start
#top_alcaldias = ['Coyoacán', 'Álvaro Obregón', 'Benito Juárez', 'Cuauhtémoc', 'Miguel Hidalgo']
#cdmx = tmp[tmp.Alcaldia.isin(top_alcaldias) & tmp.State.isin(['Ciudad de México'])]
# cdmx = tmp[tmp.State.isin(['Ciudad de México'])]

# save table
states_and_zipcodes = tmp
states_and_zipcodes.to_csv('../data/states_and_zipcodes.csv')

#### 3. Aggregate health stabishments data by location

In [423]:
# bring columns of interest
health_locations = health_locations[[
    'ID',
    'NOMBRE DE LA UNIDAD',
    'TOTAL DE CONSULTORIOS',
    'TOTAL DE CAMAS',
    'TIPO DE VIALIDAD',
    'TIPO DE ASENTAMIENTO',
    'NOMBRE DE LA ENTIDAD',
    'CODIGO POSTAL',
    'LATITUD',
    'LONGITUD',
    'ESTATUS DE OPERACION',
    'ESTRATO UNIDAD'    
    ]]

health_locations.dropna(inplace=True) # bad data quality from source -> fast approach is to drop all NA values

# format zipcodes to string 5 characters
health_locations['CODIGO POSTAL'] = health_locations['CODIGO POSTAL'].astype(int).astype(str).apply(str).str.zfill(5)

# group establishments by zip code
healthByZipCode = health_locations.groupby(['CODIGO POSTAL'])['ID'].count().reset_index(drop=False)\
                                 .rename(columns={
                                    'CODIGO POSTAL':'ZipCode',
                                    'ID':'HealthStabishmentCount',
                                    })
# bring states into dataset
healthByZipCode = pd.merge(healthByZipCode, states_and_zipcodes,  how='left', on='ZipCode')

healthByZipCode.dropna(inplace=True) # drop NA's for ease

#### 4. Aggregate count of wholesale commerce stabishments by zip code

In [424]:
# bring columns of interest
wholesale = wholesale[[
    'id',
    'nom_estab',
    'per_ocu',
    'cod_postal',
    'entidad'
    ]]

wholesale.dropna(inplace=True) # bad data quality from source -> fast approach is to drop all NA values

# format zipcodes to string 5 characters
wholesale['cod_postal'] = wholesale['cod_postal'].astype(str).apply(str).str.zfill(5)

wholesale = wholesale.groupby(['entidad', 'cod_postal']).agg({
                        'id' : 'count',
                        'per_ocu' : 'mean'
                        })\
                        .reset_index(drop=False)\
                        .rename(columns={
                                    'id':'Wholesale_StabishmentCount',
                                    'per_ocu':'Wholesale_AverageFlux',
                                    'entidad':'State',
                                    'cod_postal':'ZipCode'
                                    })

#### 5. Aggregate count of construction industry stabishments by zip code

In [425]:
# bring columns of interest
construction = construction[[
    'id',
    'nom_estab',
    'per_ocu',
    'cod_postal',
    'entidad'
    ]]

construction.dropna(inplace=True) # bad data quality from source -> fast approach is to drop all NA values

# format zipcodes to string 5 characters
construction['cod_postal'] = construction['cod_postal'].astype(str).apply(str).str.zfill(5)

construction = construction.groupby(['entidad', 'cod_postal']).agg({
                        'id' : 'count',
                        'per_ocu' : 'mean'
                        })\
                        .reset_index(drop=False)\
                        .rename(columns={
                                    'id':'Construction_StabishmentCount',
                                    'per_ocu':'Construction_AverageFlux',
                                    'entidad':'State',
                                    'cod_postal':'ZipCode'
                                    })

#### 6. Aggregate count of corporates centres by zip code

In [426]:
# bring columns of interest
corporate = corporate[[
    'id',
    'nom_estab',
    'per_ocu',
    'cod_postal',
    'entidad'
    ]]

corporate.dropna(inplace=True) # bad data quality from source -> fast approach is to drop all NA values

# format zipcodes to string 5 characters
corporate['cod_postal'] = corporate['cod_postal'].astype(str).apply(str).str.zfill(5)

corporate = corporate.groupby(['entidad', 'cod_postal']).agg({
                        'id' : 'count',
                        'per_ocu' : 'mean'
                        })\
                        .reset_index(drop=False)\
                        .rename(columns={
                                    'id':'Corporate_StabishmentCount',
                                    'per_ocu':'Corporate_AverageFlux',
                                    'entidad':'State',
                                    'cod_postal':'ZipCode'
                                    })

#### 7. Aggregate count of education centres by zip code

In [427]:
# bring columns of interest
education = education[[
    'id',
    'nom_estab',
    'per_ocu',
    'cod_postal',
    'entidad'
    ]]

education.dropna(inplace=True) # bad data quality from source -> fast approach is to drop all NA values

# format zipcodes to string 5 characters
education['cod_postal'] = education['cod_postal'].astype(str).apply(str).str.zfill(5)

education = education.groupby(['entidad', 'cod_postal']).agg({
                        'id' : 'count',
                        'per_ocu' : 'mean'
                        })\
                        .reset_index(drop=False)\
                        .rename(columns={
                                    'id':'Education_StabishmentCount',
                                    'per_ocu':'Education_AverageFlux',
                                    'entidad':'State',
                                    'cod_postal':'ZipCode'
                                    })

#### 8. Aggregate count of goverment centres by zip code

In [428]:
# bring columns of interest
goverment = goverment[[
    'id',
    'nom_estab',
    'per_ocu',
    'cod_postal',
    'entidad'
    ]]

goverment.dropna(inplace=True) # bad data quality from source -> fast approach is to drop all NA values

# format zipcodes to string 5 characters
goverment['cod_postal'] = goverment['cod_postal'].astype(str).apply(str).str.zfill(5)

goverment = goverment.groupby(['entidad', 'cod_postal']).agg({
                        'id' : 'count',
                        'per_ocu' : 'mean'
                        })\
                        .reset_index(drop=False)\
                        .rename(columns={
                                    'id':'Goverment_StabishmentCount',
                                    'per_ocu':'Goverment_AverageFlux',
                                    'entidad':'State',
                                    'cod_postal':'ZipCode'
                                    })

#### N-1. Enrich ENOE's Foodstand information with INEGI data

In [429]:
# merge health centres
df = pd.merge(foodstandsByLocation, healthByZipCode,  how='left', on='ZipCode')
#df.dropna(inplace=True)
df['HealthStabishmentCount'].fillna(value=df['HealthStabishmentCount'].mean(), inplace=True) # Input mean to NA's values

# merge wholesale centres
df = pd.merge(df, wholesale,  how='left', on='ZipCode')
#df.dropna(inplace=True)
df['Wholesale_StabishmentCount'].fillna(value=df['Wholesale_StabishmentCount'].mean(), inplace=True) # Input mean to NA's values
df['Wholesale_AverageFlux'].fillna(value=df['Wholesale_AverageFlux'].mean(), inplace=True) # Input mean to NA's values

# merge construction centres
df = pd.merge(df, construction,  how='left', on='ZipCode')
#df.dropna(inplace=True)
df['Construction_StabishmentCount'].fillna(value=df['Construction_StabishmentCount'].mean(), inplace=True) # Input mean to NA's values
df['Construction_AverageFlux'].fillna(value=df['Construction_AverageFlux'].mean(), inplace=True) # Input mean to NA's values

# merge corporate centres
df = pd.merge(df, corporate,  how='left', on='ZipCode')
#df.dropna(inplace=True)
df['Corporate_StabishmentCount'].fillna(value=df['Corporate_StabishmentCount'].mean(), inplace=True) # Input mean to NA's values
df['Corporate_AverageFlux'].fillna(value=df['Corporate_AverageFlux'].mean(), inplace=True) # Input mean to NA's values

# merge education centres
df = pd.merge(df, education,  how='left', on='ZipCode')
#df.dropna(inplace=True)
df['Education_StabishmentCount'].fillna(value=df['Education_StabishmentCount'].mean(), inplace=True) # Input mean to NA's values
df['Education_AverageFlux'].fillna(value=df['Education_AverageFlux'].mean(), inplace=True) # Input mean to NA's values

# merge goverment centres
df = pd.merge(df, goverment,  how='left', on='ZipCode')
#df.dropna(inplace=True)
df['Goverment_StabishmentCount'].fillna(value=df['Goverment_StabishmentCount'].mean(), inplace=True) # Input mean to NA's values
df['Goverment_AverageFlux'].fillna(value=df['Goverment_AverageFlux'].mean(), inplace=True) # Input mean to NA's values


C:\Users\mike_\AppData\Local\Temp\ipykernel_4136\1313774910.py:13: FutureWarning: Passing 'suffixes' which cause duplicate columns {'State_x'} in the result is deprecated and will raise a MergeError in a future version.
  df = pd.merge(df, construction,  how='left', on='ZipCode')
C:\Users\mike_\AppData\Local\Temp\ipykernel_4136\1313774910.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Construction_StabishmentCount'].fillna(value=df['Construction_StabishmentCount'].mean(), inplace=True) # Input mean to NA's values
C:\Users\mike_\AppData\Local\Temp\ipykernel_4136\1313774910.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-

#### N. Consolidate final dataframe

In [430]:
final_df = df

# Drop columns based on column index.
final_df = final_df.drop(final_df.columns[[0, 4, 6, 9, 12, 15]],axis = 1)

# arrange column order
final_df = final_df[[
                    'State',
                    'Alcaldia',
                    'ZipCode',
                    'FoodstandCount',
                    'HealthStabishmentCount',
                    'Wholesale_StabishmentCount',
                    'Wholesale_AverageFlux',
                    'Construction_StabishmentCount',
                    'Construction_AverageFlux',
                    'Corporate_StabishmentCount',
                    'Corporate_AverageFlux',
                    'Education_StabishmentCount',
                    'Education_AverageFlux',
                    'Goverment_StabishmentCount',
                    'Goverment_AverageFlux'
                    ]] 

In [436]:
final_df['Construction_StabishmentCount'].fillna(value=final_df['Construction_StabishmentCount'].mean(), inplace=True) # Input mean to NA's values
final_df['Construction_AverageFlux'].fillna(value=final_df['Construction_AverageFlux'].mean(), inplace=True) # Input mean to NA's values

final_df['Corporate_StabishmentCount'].fillna(value=final_df['Corporate_StabishmentCount'].mean(), inplace=True) # Input mean to NA's values
final_df['Corporate_AverageFlux'].fillna(value=final_df['Corporate_AverageFlux'].mean(), inplace=True) # Input mean to NA's values

final_df['Education_StabishmentCount'].fillna(value=final_df['Education_StabishmentCount'].mean(), inplace=True) # Input mean to NA's values
final_df['Education_AverageFlux'].fillna(value=final_df['Education_AverageFlux'].mean(), inplace=True) # Input mean to NA's values

final_df['Goverment_StabishmentCount'].fillna(value=final_df['Goverment_StabishmentCount'].mean(), inplace=True) # Input mean to NA's values
final_df['Goverment_AverageFlux'].fillna(value=final_df['Goverment_AverageFlux'].mean(), inplace=True) # Input mean to NA's values

